## Interval variant count

We would like to aggregate the data according to the intervals, to cehck how many variants there are in each interval <br>
The following notebook shows 3 kind of interval analysis:
1. by GH associated DSD genes
2. by DSD genes which are 1mb from an interval
3. by genes which are 1.5mb from an interval

In each analysis, 3 subgroups will be analysed - 
1. all samples
2. samples diagnosed with DSD

Ken's data and Sinclair data will be analysed seperately

In [1]:

import sys
import os
sys.path.append(os.path.abspath("code"))
# these are  needed-
import pandas as pd
from GnomAD_df_class import GnomAD_df


In [47]:
gdf = GnomAD_df(path='data/main_new_AF.prq',
                peak_file='data/hATAC_mATAC_peak.bed.gz',
                remove_unkown=True,
                remove_phased_gt=True,
                only_peak_variants=True)
sinclair_sample_data = pd.read_csv('data/pedigree_data.tsv', sep='\t')
sin_probands = sinclair_sample_data.id[sinclair_sample_data.fam_relation == 0].tolist() 
sin_non_probands = sinclair_sample_data.id[~sinclair_sample_data.id.isin(probands)] 
ken_probands =   [i for i in gdf.get_samples() if i.startswith('P') and i != 'PKGen169']
ken_non_probands = ['PKGen169']
interval_df = pd.read_csv('data/intervals_with_overlaps_1_1.5mb.csv', index_col=0).set_index('INTERVAL_ID')
# gdf.get_table().head()

We will start with sinclair data <br>
first, we need to filter the main table to match the samples.

In [20]:
print(f'Sincalair data contains {len(sin_probands)} samples with DSD and \
{len(sin_non_probands)} without \n total- {len(sin_probands) +len(sin_non_probands)} variants')


Sincalair data contains 9 samples with DSD and 15 without 
 total- 24 variants


In [28]:
gdf.filter_samples([i for i in gdf.get_samples() if i not in sin_probands + sin_non_probands.tolist() ])
bool_df = gdf.bool_variant_df()
bool_df.head()

Filtering samples
removing variants outside of peak interval
removed unkwon genotypes (e.g "./.")
replaced phased genotype (e.g "0|1" -> "0/1")
done


NEW_AF          INTERVAL_ID  AS22WG009  AS22WG010  \
CHROM POS    REF ALT                                                        
chr1  634112 T   C         NaN       PGRA.SER.GRA_1       True       True   
      778639 A   G    0.127933  PGRA.SER.GRA.PSER_2      False      False   
      779047 G   A    0.819806  PGRA.SER.GRA.PSER_2       True       True   
      826893 G   A    0.730133  PGRA.SER.GRA.PSER_3       True       True   
      827209 G   C    0.739536  PGRA.SER.GRA.PSER_3       True       True   

                      AS22WG011  AS22WG023  AS22WG024  AS22WG004  AS22WG005  \
CHROM POS    REF ALT                                                          
chr1  634112 T   C         True       True       True       True       True   
      778639 A   G        False      False      False      False      False   
      779047 G   A         True       True       True       True       True   
      826893 G   A         True       True       True       True       True   
      827209 G   C         True       True       True       True       True   

                      AS22WG006  ...  AS22WG013  AS22WG014  AS22WG015  \
CHROM POS    REF ALT             ...                                    
chr1  634112 T   C         True  ...       True       True       True   
      778639 A   G        False  ...      False      False      False   
      779047 G   A         True  ...       True       True       True   
      826893 G   A         True  ...       True       True       True   
      827209 G   C         True  ...       True       True       True   

                      AS22WG016  AS22WG017  AS22WG018  AS22WG019  AS22WG020  \
CHROM POS    REF ALT                                                          
chr1  634112 T   C         True       True       True       True       True   
      778639 A   G        False      False      False      False      False   
      779047 G   A         True       True       True       True       True   
      826893 G   A         True       True       True       True       True   
      827209 G   C         True       True       True       True       True   

                      AS22WG021  AS22WG022  
CHROM POS    REF ALT                        
chr1  634112 T   C         True       True  
      778639 A   G        False      False  
      779047 G   A         True       True  
      826893 G   A         True       True  
      827209 G   C         True       True  

[5 rows x 26 columns]

In [54]:
def get_numbers(bool_df,probands, non_probands, prefix='', AF_t=1):
    bool_df = bool_df[bool_df.NEW_AF <= AF_t]
    all_samples = bool_df.drop(columns='NEW_AF').groupby('INTERVAL_ID').sum().sum(axis=1).rename(prefix + 'all_samples')
    only_probands = bool_df.drop(columns=['NEW_AF'] + non_probands).groupby('INTERVAL_ID').sum().sum(axis=1).rename(prefix + 'only_probands')
    only_non_probands = bool_df.drop(columns=['NEW_AF'] + probands).groupby('INTERVAL_ID').sum().sum(axis=1).rename(prefix +'only_non_probands')
    return pd.DataFrame(data=[all_samples, only_probands, only_non_probands]).T

# sin_af1  = get_numbers(bool_df, sin_probands, sin_non_probands.tolist(),'sin_').sort_values('sin_all_samples', ascending=False)
# sin_aff =  get_numbers(bool_df, sin_probands, sin_non_probands.tolist(),'sin_af<0.01_', AF_t=0.01).sort_values('sin_af<0.01_all_samples', ascending=False)
# sin_aff.head()

In [64]:
sinclair = pd.concat([interval_df,sin_af1,sin_aff], axis=1).sort_values('sin_af<0.01_all_samples', ascending=False)

In [65]:
# gdf.reset_table().filter_samples([i for i in gdf.get_samples() if i not in ken_probands + ken_non_probands ])
# bool_df = gdf.bool_variant_df()
# bool_df.head()

ken_af1  = get_numbers(bool_df, ken_probands, ken_non_probands,'ken_')
ken_aff =  get_numbers(bool_df, ken_probands, ken_non_probands,'ken_af<0.01_', AF_t=0.01)
ken_aff.head()

,ken_af<0.01_all_samples,ken_af<0.01_only_probands,ken_af<0.01_only_non_probands
INTERVAL_ID,,,
CM100017,1,1,0
CM100097,1,1,0
CM100099,1,0,1
CM100102,1,1,0
CM100109,1,1,0


In [62]:
ken

,CHROM,FROM,TO,GH_DSD_genes,GH_DSD,dsd_genes1mb,dsd_gene1.5mb,all_genes1.5mb,num_of_genes,ken_all_samples,ken_only_probands,ken_only_non_probands,ken_af<0.01_all_samples,ken_af<0.01_only_probands,ken_af<0.01_only_non_probands
INTERVAL_ID,,,,,,,,,,,,,,,
PGRA.SER_6229,chr11,1015808,1017938,NaN,False,NaN,IGF2,WBP1LP10_OR4F2P_ENSG00000254468_CICP23_LINC010...,132,74.0,69.0,5.0,54.0,53.0,1.0
mATAC_hATAC_32167,chr6,31355973,31357525,NaN,False,NaN,NaN,ENSG00000290870_HCP5B_ENSG00000230521_HLA-H_HL...,210,66.0,60.0,6.0,40.0,37.0,3.0
mATAC_hATAC_32222,chr6,32583486,32584897,NaN,False,NaN,NaN,C6orf15_PSORS1C1_CDSN_PSORS1C2_POLR2LP1_CCHCR1...,186,77.0,75.0,2.0,36.0,34.0,2.0
mATAC_hATAC_32164,chr6,31270881,31272606,NaN,False,NaN,NaN,HLA-V_HCG4_HLA-V_HLA-P_RPL7AP7_MICG_HLA-G_HCG4...,215,192.0,171.0,21.0,35.0,31.0,4.0
PGRA.SER.PSER.GRA_28538,chr3,75667918,75670704,NaN,False,NaN,NaN,CNTN3_NIPA2P2_RN7SL294P_MIR4444-2_ENSG00000287...,41,69.0,66.0,3.0,32.0,31.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SER_42024,chrY,19743944,19745337,NaN,False,NaN,NaN,TAF9P2_CLUHP2_FAM224A_ENSG00000278391_RNA5SP52...,33,NaN,NaN,NaN,NaN,NaN,NaN
SER.PSER_42025,chrY,20574998,20576299,NaN,False,NaN,NaN,TTTY14_ENSG00000274282_GAPDHP19_BCORP1_BCORP1_...,38,NaN,NaN,NaN,NaN,NaN,NaN
CM189619,chrY,20753080,20753096,NaN,False,NaN,NaN,ENSG00000274282_GAPDHP19_BCORP1_BCORP1_TXLNGY_...,38,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
ken = pd.concat([interval_df,ken_af1,ken_aff], axis=1).sort_values('ken_af<0.01_all_samples', ascending=False)

In [67]:

# create a excel writer object
with pd.ExcelWriter("outputs/interval_variants.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    sinclair.to_excel(writer, sheet_name="sinclair")
    ken.to_excel(writer, sheet_name="ken")

,INTERVAL_ID,CHROM,FROM,TO,GH_DSD_genes,GH_DSD,dsd_genes1mb,dsd_gene1.5mb,all_genes1.5mb,num_of_genes,sin_all_samples,sin_only_probands,sin_only_non_probands,sin_af<0.01_all_samples,sin_af<0.01_only_probands,sin_af<0.01_only_non_probands
0,PGRA.SER_6229,chr11,1015808,1017938,NaN,False,NaN,IGF2,WBP1LP10_OR4F2P_ENSG00000254468_CICP23_LINC010...,132,70.0,32.0,38.0,38.0,15.0,23.0
1,PGRA.SER.PSER.GRA_28538,chr3,75667918,75670704,NaN,False,NaN,NaN,CNTN3_NIPA2P2_RN7SL294P_MIR4444-2_ENSG00000287...,41,122.0,53.0,69.0,33.0,10.0,23.0
2,PGRA.SER.GRA.PSER_39577,chr9,62837274,62838738,NaN,False,NaN,NaN,CNTNAP3C_RN7SL462P_ENSG00000277109_FAM242D_Y_R...,65,79.0,32.0,47.0,31.0,14.0,17.0
3,mATAC_hATAC_22657,chr2,178193444,178195490,NaN,False,NaN,NaN,LINC01117_ENSG00000230552_FUCA1P1_ENSG00000223...,52,68.0,26.0,42.0,25.0,10.0,15.0
4,mATAC_hATAC_9960,chr12,108855616,108858272,['SART3 : 23.5516'],True,SART3,SART3,ABTB3_ENSG00000200897_RNA5SP371_ENSG0000025757...,53,33.0,13.0,20.0,3.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193675,SER_42024,chrY,19743944,19745337,NaN,False,NaN,NaN,TAF9P2_CLUHP2_FAM224A_ENSG00000278391_RNA5SP52...,33,NaN,NaN,NaN,NaN,NaN,NaN
193676,SER.PSER_42025,chrY,20574998,20576299,NaN,False,NaN,NaN,TTTY14_ENSG00000274282_GAPDHP19_BCORP1_BCORP1_...,38,NaN,NaN,NaN,NaN,NaN,NaN
193677,CM189619,chrY,20753080,20753096,NaN,False,NaN,NaN,ENSG00000274282_GAPDHP19_BCORP1_BCORP1_TXLNGY_...,38,NaN,NaN,NaN,NaN,NaN,NaN
193678,CM189620,chrY,57183942,57184105,NaN,False,NaN,NaN,CTBP2P1_AMD1P2_DPH3P2_VAMP7_ELOCP24_TRPC6P1_IL...,11,NaN,NaN,NaN,NaN,NaN,NaN
